1) Download all symbols and their weights in the S&P 500.

In [3]:
import pandas as pd
import requests
import cloudscraper
from io import StringIO
from bs4 import BeautifulSoup

url = "https://www.slickcharts.com/sp500"
scraper = cloudscraper.create_scraper()
soup = BeautifulSoup(scraper.get(url).text, 'html.parser')
sap_table = soup.find('table', {'class':"table"})
sap_list = pd.read_html(StringIO(str(sap_table)), flavor='html5lib')[0]
sap = pd.DataFrame(sap_list)

sap = sap[["Company", "Symbol", "Portfolio%"]]
sap

,Company,Symbol,Portfolio%
0,Microsoft Corp,MSFT,7.14%
1,Apple Inc.,AAPL,6.08%
2,Nvidia Corp,NVDA,4.70%
3,Amazon.com Inc,AMZN,3.75%
4,"Meta Platforms, Inc. Class A",META,2.58%
...,...,...,...
498,Zions Bancorporation N.a.,ZION,0.01%
499,V.F. Corporation,VFC,0.01%
500,Paramount Global Class B,PARA,0.01%
501,Fox Corporation Class B,FOX,0.01%


2) Remove any equities you don't want to hold

In [4]:
banned_tickers = ("TSLA", "FOX", "NWS")
sap = sap[~sap["Symbol"].isin(banned_tickers)].copy()
sap

,Company,Symbol,Portfolio%
0,Microsoft Corp,MSFT,7.14%
1,Apple Inc.,AAPL,6.08%
2,Nvidia Corp,NVDA,4.70%
3,Amazon.com Inc,AMZN,3.75%
4,"Meta Platforms, Inc. Class A",META,2.58%
...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%
497,Whirlpool Corp.,WHR,0.01%
498,Zions Bancorporation N.a.,ZION,0.01%
499,V.F. Corporation,VFC,0.01%


3. Also optionally clip weights

In [5]:
sap["PortfolioFraction"] = sap["Portfolio%"].str.strip("%").astype('float') / 100
sap["PortfolioFraction"] = sap["PortfolioFraction"].clip(lower=0, upper=0.05)
sap

,Company,Symbol,Portfolio%,PortfolioFraction
0,Microsoft Corp,MSFT,7.14%,0.0500
1,Apple Inc.,AAPL,6.08%,0.0500
2,Nvidia Corp,NVDA,4.70%,0.0470
3,Amazon.com Inc,AMZN,3.75%,0.0375
4,"Meta Platforms, Inc. Class A",META,2.58%,0.0258
...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.0001
497,Whirlpool Corp.,WHR,0.01%,0.0001
498,Zions Bancorporation N.a.,ZION,0.01%,0.0001
499,V.F. Corporation,VFC,0.01%,0.0001


4. Reweight now that some of the tickers were removed or clipped

In [6]:
increase_prec = 1 - sap["PortfolioFraction"].sum() + 1
sap["PortfolioFraction"] = sap["PortfolioFraction"] * increase_prec
sap

,Company,Symbol,Portfolio%,PortfolioFraction
0,Microsoft Corp,MSFT,7.14%,0.052395
1,Apple Inc.,AAPL,6.08%,0.052395
2,Nvidia Corp,NVDA,4.70%,0.049251
3,Amazon.com Inc,AMZN,3.75%,0.039296
4,"Meta Platforms, Inc. Class A",META,2.58%,0.027036
...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.000105
497,Whirlpool Corp.,WHR,0.01%,0.000105
498,Zions Bancorporation N.a.,ZION,0.01%,0.000105
499,V.F. Corporation,VFC,0.01%,0.000105


5) Import a dataframe of your current equity holdings & cash. You're a bit on your own here but my brokerage allows me to download a CSV of all of my holding. I'm going to assume you have a simplified version of that.

In [7]:
#Write out some test data
with open('holdings.csv', 'w') as f:
    f.write("""symbol,shares
AMZN,10
NVDA,1
MSFT,1
YUM,100
ZTS,10
GME,100""")

In [8]:
holdings = pd.read_csv('holdings.csv')
holdings

,symbol,shares
0,AMZN,10
1,NVDA,1
2,MSFT,1
3,YUM,100
4,ZTS,10
5,GME,100


In [9]:
cash_to_invest = 10000

6) Get last share price of all of holdings and compute our total assests to be invested

In [14]:
import yfinance as yf

def get_current_price(symbol):
    ticker = yf.Ticker(symbol.replace(".","-"))
    todays_data = ticker.history(period='2d')
    return todays_data['Close'].iloc[0]

holdings["last$"] = holdings["symbol"].apply(get_current_price)
holdings["total$"] = holdings["shares"] * holdings["last$"]

total_assets = holdings['total$'].sum() + cash_to_invest
total_assets
print(f"total_assets:", total_assets)
holdings

total_assets: 30298.690364837646


,symbol,shares,last$,total$
0,AMZN,10,177.580002,1775.800018
1,NVDA,1,852.369995,852.369995
2,MSFT,1,414.920013,414.920013
3,YUM,100,138.550003,13855.000305
4,ZTS,10,187.860001,1878.600006
5,GME,100,15.220000,1522.000027


7) Also get lastest share price from all S&P 500 symbols

In [16]:
sap["last$"] = sap["Symbol"].apply(get_current_price)

In [17]:
sap

,Company,Symbol,Portfolio%,PortfolioFraction,last$
0,Microsoft Corp,MSFT,7.14%,0.052395,414.920013
1,Apple Inc.,AAPL,6.08%,0.052395,175.100006
2,Nvidia Corp,NVDA,4.70%,0.049251,852.369995
3,Amazon.com Inc,AMZN,3.75%,0.039296,177.580002
4,"Meta Platforms, Inc. Class A",META,2.58%,0.027036,498.190002
...,...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.000105,120.830002
497,Whirlpool Corp.,WHR,0.01%,0.000105,106.550003
498,Zions Bancorporation N.a.,ZION,0.01%,0.000105,39.169998
499,V.F. Corporation,VFC,0.01%,0.000105,15.730000


8) Compute shares we should own to match give our current investable assets

In [18]:
sap["$NeededToMatch"] = sap["PortfolioFraction"] * total_assets
sap["SharesToMatch"] = (sap["$NeededToMatch"] / sap["last$"]).astype('int') # assume no factional shares
sap

,Company,Symbol,Portfolio%,PortfolioFraction,last$,$NeededToMatch,SharesToMatch
0,Microsoft Corp,MSFT,7.14%,0.052395,414.920013,1587.499882,3
1,Apple Inc.,AAPL,6.08%,0.052395,175.100006,1587.499882,9
2,Nvidia Corp,NVDA,4.70%,0.049251,852.369995,1492.249889,1
3,Amazon.com Inc,AMZN,3.75%,0.039296,177.580002,1190.624911,6
4,"Meta Platforms, Inc. Class A",META,2.58%,0.027036,498.190002,819.149939,1
...,...,...,...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.000105,120.830002,3.175000,0
497,Whirlpool Corp.,WHR,0.01%,0.000105,106.550003,3.175000,0
498,Zions Bancorporation N.a.,ZION,0.01%,0.000105,39.169998,3.175000,0
499,V.F. Corporation,VFC,0.01%,0.000105,15.730000,3.175000,0


9) Join to our current holdings

In [19]:
pd.set_option('display.max_rows', None)
movesWithHoldings = sap.merge(holdings[["symbol", "shares"]], left_on='Symbol', right_on='symbol', how='outer')
movesWithHoldings["shares"] = movesWithHoldings["shares"].fillna(0)
movesWithHoldings["SharesToMatch"] = movesWithHoldings["SharesToMatch"].fillna(0)
movesWithHoldings.loc[movesWithHoldings["Symbol"].isna(),"Symbol"] = movesWithHoldings[movesWithHoldings["Symbol"].isna()]["symbol"]
movesWithHoldings

,Company,Symbol,Portfolio%,PortfolioFraction,last$,$NeededToMatch,SharesToMatch,symbol,shares
0,Microsoft Corp,MSFT,7.14%,0.052395,414.920013,1587.499882,3.0,MSFT,1.0
1,Apple Inc.,AAPL,6.08%,0.052395,175.100006,1587.499882,9.0,NaN,0.0
2,Nvidia Corp,NVDA,4.70%,0.049251,852.369995,1492.249889,1.0,NVDA,1.0
3,Amazon.com Inc,AMZN,3.75%,0.039296,177.580002,1190.624911,6.0,AMZN,10.0
4,"Meta Platforms, Inc. Class A",META,2.58%,0.027036,498.190002,819.149939,1.0,NaN,0.0
5,Alphabet Inc. Class A,GOOGL,1.88%,0.019701,133.350006,596.899956,4.0,NaN,0.0
6,Berkshire Hathaway Class B,BRK.B,1.71%,0.017919,403.390015,542.924960,1.0,NaN,0.0
7,Alphabet Inc. Class C,GOOG,1.59%,0.016662,134.199997,504.824962,3.0,NaN,0.0
8,Eli Lilly & Co.,LLY,1.44%,0.015090,792.280029,457.199966,0.0,NaN,0.0
9,Broadcom Inc.,AVGO,1.42%,0.014880,1402.260010,450.849966,0.0,NaN,0.0


11) Find the shares we need to match the S&P 500 prop

In [20]:
movesWithHoldings["SharesDifference"] = movesWithHoldings["SharesToMatch"] - movesWithHoldings["shares"]
movesWithHoldings

,Company,Symbol,Portfolio%,PortfolioFraction,last$,$NeededToMatch,SharesToMatch,symbol,shares,SharesDifference
0,Microsoft Corp,MSFT,7.14%,0.052395,414.920013,1587.499882,3.0,MSFT,1.0,2.0
1,Apple Inc.,AAPL,6.08%,0.052395,175.100006,1587.499882,9.0,NaN,0.0,9.0
2,Nvidia Corp,NVDA,4.70%,0.049251,852.369995,1492.249889,1.0,NVDA,1.0,0.0
3,Amazon.com Inc,AMZN,3.75%,0.039296,177.580002,1190.624911,6.0,AMZN,10.0,-4.0
4,"Meta Platforms, Inc. Class A",META,2.58%,0.027036,498.190002,819.149939,1.0,NaN,0.0,1.0
5,Alphabet Inc. Class A,GOOGL,1.88%,0.019701,133.350006,596.899956,4.0,NaN,0.0,4.0
6,Berkshire Hathaway Class B,BRK.B,1.71%,0.017919,403.390015,542.924960,1.0,NaN,0.0,1.0
7,Alphabet Inc. Class C,GOOG,1.59%,0.016662,134.199997,504.824962,3.0,NaN,0.0,3.0
8,Eli Lilly & Co.,LLY,1.44%,0.015090,792.280029,457.199966,0.0,NaN,0.0,0.0
9,Broadcom Inc.,AVGO,1.42%,0.014880,1402.260010,450.849966,0.0,NaN,0.0,0.0


12) Find tickers that we need to sell

In [21]:
movesWithHoldings[movesWithHoldings["SharesDifference"] < 0][["Symbol", "SharesDifference"]] 

,Symbol,SharesDifference
3,AMZN,-4.0
104,ZTS,-10.0
220,YUM,-100.0
500,GME,-100.0


13) Find tickers that we need to buy

In [22]:
movesWithHoldings[movesWithHoldings["SharesDifference"] > 0][["Symbol", "SharesDifference"]]

,Symbol,SharesDifference
0,MSFT,2.0
1,AAPL,9.0
4,META,1.0
5,GOOGL,4.0
6,BRK.B,1.0
7,GOOG,3.0
10,JPM,2.0
12,V,1.0
13,XOM,2.0
15,JNJ,1.0
